In [4]:
from pathlib import Path
from typing import Optional, List, NamedTuple, Tuple
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.float_format = '{:,.0f}'.format

In [5]:
class Run(NamedTuple):
    alg_name: str
    arch: str
    params: Tuple[Tuple[str, str]]
    path: Path
    runtime: Optional[str]
    durs: np.array
    mean: float

    @classmethod
    def from_path(cls, path: Path):
        tokens = path.name.split("!")
        alg_name = tokens[0]
        arch = tokens[1]
        runtime = "native"
        if arch == "wasm_single" or arch == "wasm_multi":
            runtime = tokens.pop()
        params = dict()
        for pair in tokens[2:]:
            items = pair.split("+", maxsplit=1)
            params[items[0]] = items[1]

        durs: List[float] = list()
        with open(path) as f:
            for l in f:
                durs.append(float(l.strip()))

        durs = np.array(durs)

        return cls(
            alg_name,
            arch,
            tuple(params.items()),
            path,
            runtime,
            durs,
            durs.mean(),
        )

In [6]:
runs: List[Run] = list()

log_path = Path("logs")
for p in log_path.iterdir():
    if p.name == "err":
        continue
    r = Run.from_path(p)
    runs.append(r)

df = pd.DataFrame(runs)
pt = pd.pivot_table(df,
    index=["alg_name","params", "runtime"],
    values=["mean"])
pt

mean
alg_name     params                                             runtime             
matrix_naive ((a, 2000), (b, 1200), (c, 1500), (seed, 13), (... firefox       51,799
                                                                google-chrome 63,098
                                                                native         5,581
             ((a, 2000), (b, 1200), (c, 1500), (seed, 13), (... firefox       30,354
                                                                google-chrome 40,584
                                                                native         2,816
             ((a, 2000), (b, 1200), (c, 1500), (seed, 13), (... firefox       18,513
                                                                google-chrome 30,308
                                                                native         1,393
             ((a, 999), (b, 888), (c, 777), (seed, 13))         firefox        9,268
                                                                google-chrome 10,824
                                                                native           771
             ((a, 999), (b, 888), (c, 777), (seed, 13), (num... firefox        9,171
                                                                google-chrome 11,634
                                                                native           769
             ((a, 999), (b, 888), (c, 777), (seed, 13), (num... firefox        7,399
                                                                google-chrome  9,474
                                                                native           421
             ((a, 999), (b, 888), (c, 777), (seed, 13), (num... firefox        2,556
                                                                google-chrome  2,846
                                                                native           209
merge        ((size, 100000), (n_levels, 0))                    firefox           54
                                                                google-chrome     79
                                                                native            16
             ((size, 100000), (n_levels, 1))                    firefox           24
                                                                google-chrome     31
                                                                native            10
             ((size, 100000), (n_levels, 2))                    firefox           17
                                                                google-chrome     25
                                                                native             6
             ((size, 1000000), (n_levels, 0))                   firefox          342
                                                                google-chrome    514
                                                                native           147
             ((size, 1000000), (n_levels, 1))                   firefox          206
                                                                google-chrome    286
                                                                native            77
             ((size, 1000000), (n_levels, 2))                   firefox          132
                                                                google-chrome    182
                                                                native            49

In [6]:
pt.to_excel("output.xlsx")